# 1. 해당 태그를 가지는 유튜브 영상 가져오기

In [1]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = [ "https://www.googleapis.com/auth/youtube",
"https://www.googleapis.com/auth/youtube.force-ssl",
"https://www.googleapis.com/auth/youtube.readonly",
"https://www.googleapis.com/auth/youtubepartner"]

### Google OAuth 2.0 등록

In [2]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "../secret_key/client_secret.json"
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=699806263406-koq1jouhe37k38dvvshpu98ib9m5o53g.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutubepartner&state=H8QGgdqht5pXgyYvsjIAx4xL4SoW8E&prompt=consent&access_type=offline
Enter the authorization code: 4/ywGsO3vokOH6Aw_Tu4-rkTYdSrmvfLohLmsDEFOWerK2Qgceu7_gFA0


In [3]:
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

## opic 키워드의 상위 50개 영상 video id 가져오기

In [110]:
MAX_RESULTS=50
query = "opic"
search_response = youtube.search().list(
    part="id,snippet",
    q=query,
    type='video',
    maxResults=MAX_RESULTS,
    pageToken='',
  ).execute()

In [111]:
video_ids = []
VIDEO = 'youtube#video'
for i in range(len(search_response['items'])):
    if search_response['items'][i]['id']['kind'] == VIDEO:
        video_ids.append(search_response['items'][i]['id']['videoId'])

In [112]:
','.join(video_ids[:5])

'FHpfb6b4jUM,Se4X7UWUpLU,_Cpn6_HjuFg,kfI2wOKhfvI,-B1tX7yZjSA'

In [113]:
len(video_ids)

50

## Video에 붙어있는 태그 가져오기

In [78]:
video_response = youtube.videos().list(
    part="snippet",
    id=','.join(video_ids),
  ).execute()

In [87]:
tag_dict = {}
for i in range(len(video_response['items'])):
    if video_response['items'][i]['kind'] == VIDEO:
        if 'tags' in video_response['items'][i]['snippet']:
            tags = video_response['items'][i]['snippet']['tags']
            for tag in tags:
                if tag in tag_dict:
                    tag_dict[tag] += 1
                else:
                    tag_dict[tag] = 1

In [89]:
import pandas as pd

In [105]:
df_tag = pd.DataFrame(list(tag_dict.items()), columns=['tag','count'])
df_tag.sort_values(by=['count'], inplace=True, ascending=False)
df_tag.reset_index(drop=True, inplace=True)

In [107]:
df_tag.to_csv("opic_tag.csv")

## 함수로 만들기

In [ ]:
# 기본 api 권한 설정
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = [ "https://www.googleapis.com/auth/youtube",
"https://www.googleapis.com/auth/youtube.force-ssl",
"https://www.googleapis.com/auth/youtube.readonly",
"https://www.googleapis.com/auth/youtubepartner"]

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "../secret_key/client_secret.json"
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

In [115]:
from time import sleep

In [136]:
# 검색어의 상위 50개의 video
def request_search(youtube, query):
    MAX_RESULTS=50
    sleep(0.25)
    search_response = youtube.search().list(
        part="id,snippet",
        q=query,
        type='video',
        maxResults=MAX_RESULTS,
        pageToken='',
      ).execute()

    return search_response

# 검색 결과에서 video id 뽑기
def get_video_ids(search_response):
    VIDEO = 'youtube#video'
    video_ids = []
    for i in range(len(search_response['items'])):
        if search_response['items'][i]['id']['kind'] == VIDEO:
            video_ids.append(search_response['items'][i]['id']['videoId'])

    return video_ids

# video 정보 요청
def request_videos(video_ids):
    sleep(0.25)
    video_response = youtube.videos().list(
        part="snippet",
        id=','.join(video_ids),
    ).execute()
    
    return video_response


def get_tag_counts(video_response):
    VIDEO = 'youtube#video'
    tag_dict = {}
    for i in range(len(video_response['items'])):
        if video_response['items'][i]['kind'] == VIDEO:
            if 'tags' in video_response['items'][i]['snippet']:
                tags = video_response['items'][i]['snippet']['tags']
#                 count tag
                for tag in tags:
                    if tag in tag_dict:
                        tag_dict[tag] += 1
                    else:
                        tag_dict[tag] = 1
    tag_counts = list(tag_dict.items())
    tag_counts.sort(key=lambda x: x[1], reverse=True)
    
    return tag_counts

def get_tag_df(tag_counts):
    df_tag = pd.DataFrame(tag_counts, columns=['tag','count'])
    return df_tag

In [138]:
search_response = request_search(youtube, '나니의 젊은이 일기')

In [139]:
video_ids = get_video_ids(search_response)
video_response = request_videos(video_ids)
tag_counts = get_tag_counts(video_response)
df_tag = get_tag_df(tag_counts)

In [143]:
df_tag.to_csv("나니의 젊은이 일기 tag count.csv")

In [146]:
video_ids[:5]

['D04kDsvexbM', 'HtAFAwQjOd8', 'BLRsI8x3eIY', 'e-4ZuKOgv5w', 'tLDReWJD6UE']